In [21]:
import os 
import os.path as osp 
from collections import namedtuple

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

sns.set_theme()

import utils

In [2]:
categoryPath = osp.join(".","Action_category_eng.xlsx")
df_category = utils.read_excel(categoryPath, drop_nan=False)
df_category

,category1,category2,category3,category4,category5,category6
num,,,,,,
1,eye/eyebrow,covered face,eye/eyebrow,noding head,noding head,action with chair
2,nose,covered trunk,nose/philtrum,frowning face,frowning face,touching the body
3,mouth/lips,not covered,mouth/lips/tongue,shaping hands/arms,shaking head,touching objects
4,cheek,etc,cheek,laugh/smile,laugh/smile,action with desk
5,neck,NaN,neck,shaking shoulder,yawn,yawn
6,chin,NaN,ear,oh/surprised,surprised,out of sight
7,hand,NaN,forehead,speak,tongue out,interaction_something
8,arm/shoulder,NaN,chin,tilting head,tilting head,face_etc
9,body_trunk,NaN,face,point something,crossing arms,no_face_bodyparts_etc


In [3]:
path_anno = osp.join("re_anno", "100_anno.csv")
anno_df = utils.read_csv(Path = path_anno, index_col=False)
anno_df

,name,indiv_feature,global_feature,motion,category1,category2,category3,category4,category5,category6
0,100_s0001,-1.613804,0.575877,얼굴 찡그림,4,512,8,1024,1024,32
1,100_s0002,-1.214109,-0.684512,의자 흔듬,8,512,1,2,2,2048
2,100_s0003,-0.408723,-1.167273,의자 흔듬,8,512,1,2,2,2048
3,100_s0004,-0.016706,-0.623141,의자 흔듬,8,512,1,2,2,2048
4,100_s0005,-0.025956,-0.790217,자세 바로잡기 + 얼굴 찡그림,6,512,10,18,1028,40
...,...,...,...,...,...,...,...,...,...,...
345,100_s0346,0.821566,-1.295130,고개 움직임,128,512,128,2,4,16
346,100_s0347,0.663342,-1.030960,하품함,4,512,8,2,128,128
347,100_s0348,1.188288,-0.905179,팔 & 어깨 움직임,16,512,2,2,4,8
348,100_s0349,1.264141,-0.872290,하품함,4,512,8,2,128,128


In [4]:
# (ref) https://www.educative.io/edpresso/how-to-delete-a-column-in-pandas
anno_df.pop("category2")
anno_df

,name,indiv_feature,global_feature,motion,category1,category3,category4,category5,category6
0,100_s0001,-1.613804,0.575877,얼굴 찡그림,4,8,1024,1024,32
1,100_s0002,-1.214109,-0.684512,의자 흔듬,8,1,2,2,2048
2,100_s0003,-0.408723,-1.167273,의자 흔듬,8,1,2,2,2048
3,100_s0004,-0.016706,-0.623141,의자 흔듬,8,1,2,2,2048
4,100_s0005,-0.025956,-0.790217,자세 바로잡기 + 얼굴 찡그림,6,10,18,1028,40
...,...,...,...,...,...,...,...,...,...
345,100_s0346,0.821566,-1.295130,고개 움직임,128,128,2,4,16
346,100_s0347,0.663342,-1.030960,하품함,4,8,2,128,128
347,100_s0348,1.188288,-0.905179,팔 & 어깨 움직임,16,2,2,4,8
348,100_s0349,1.264141,-0.872290,하품함,4,8,2,128,128


***

In [33]:
anno_df.iloc[0][-5:]

category1       4
category3       8
category4    1024
category5    1024
category6      32
Name: 0, dtype: object

In [30]:
def non_attention_segment(category:int, label_decimal:int) -> bool:

    non_attention_dict = {  0 : [0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0], # category1 = {noise, cheek, neck, hand, face}
                            1 : [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0], # category3 = {eye/eyebrow, nose/philtrum, mouth/lips/tongue, ear,  hair}
                            2 : [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0], # category4 = {noding head, frowning face, laugh/smile, speak, point something}
                            3 : [1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0], # category5 = {noding head, yawn, tongue out, tilting head, etc}
                            4 : [1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0], # category6 = {action_with_chair, touching the body, touching_objects, yawn, interaction_something}
                        }

    # ==== decimal to binary 
    # (ref) https://numpy.org/doc/stable/reference/generated/numpy.binary_repr.html
    # (ref) https://shayete.tistory.com/entry/%EB%A6%AC%EC%8A%A4%ED%8A%B8%EC%9D%98-%EB%AC%B8%EC%9E%90%EC%97%B4%EC%9D%84-int-%ED%98%95%ED%83%9C%EB%A1%9C-%EB%B3%80%ED%99%98
    binary = list(map(int, list(np.binary_repr(label_decimal, width=12))))

    check_act = np.bitwise_and(non_attention_dict[category],  binary)

    result = np.sum(check_act, axis=0) # (ref)http://taewan.kim/post/numpy_sum_axis/

    return bool(result)


In [42]:
action_segment = [ non_attention_segment(category, label_decimal) for category, label_decimal  in enumerate(anno_df.iloc[0][-5:])]

print(action_segment)

[True, False, True, False, True]


`action_segment` returns `[True, False, True, False, True]`. <br/>
`True` means the target action is included in the label. <br/>


예를 들어 현재 100번 피험자의 첫 번째 비디오 세그먼트에서는: 
* category1 : face 
* category3 : face 
* category4 : frowning face 
* category5 : frowning face 
* category6 : interaction_something

으로 사전에 정의된 `non_attention_segment()` 의 각 카테고리별 행동을 보면 `True` 로 판별된 카테고리에는 해당 행동이 포함되고 `False` 이면 이와는 반대로 포함되지 않는다.

(Q) `[True, False, True, False, True]` 를 non-attention 으로 볼 것인가? 
* hard decision 이면 모두 True 일때 참이므로 해당 경우에는 non-attention이 아니다. 
* soft decision 이면 `3/5` 이 `True`로 과반이 넘었기 때문에 non-attention으로 분류할 수 있다. 
* 이는 사용자 재량에 맡긴다 


In [49]:
ratio = np.sum(action_segment) / len(action_segment)

print(f"'{anno_df.iloc[0][0]}' segment is not non-attention as the ratio = {ratio}") # hard decision 
print(f"'{anno_df.iloc[0][0]}' segment is non-attention as the ratio = {ratio}") # soft decision

'100_s0001' segment is not non-attention as the ratio = 0.6
'100_s0001' segment is non-attention as the ratio = 0.6
